In [2]:
import re
import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

ModuleNotFoundError: No module named 'nltk'

In [3]:
# import text files 
path = '../data/TEXTO_CORREGIDO/*.txt'
files = glob.glob(path)

In [3]:
# creates a list from txt files
# NEED TO MAKE PROCESS TO CHECK IF TXT FILE IS WELL CONFIGURED
doc_all = []
for file in files:
    doc = [x.strip() for x in open(file, encoding='utf-8-sig')]
    doc.append(re.findall('[A-Z]{3}[0-9]{8}.', file)[0])
    doc_all.append(doc)

In [6]:
# create a pandas df and clean it
data = pd.DataFrame(data = doc_all,
                    columns = ("gen", "pos", "neg", "cam", "id"))

cl_data = data.melt(id_vars='id',
          value_vars=['gen','pos','neg','cam'])

cl_data.to_pickle('../data/data.pkl')

In [7]:
# word cloud function

def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    wordcloud = WordCloud(stopwords=stopwords_set,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(words)
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [8]:
pos = cl_data[cl_data['variable'].isin(['pos','gen'])].value
neg = cl_data[cl_data['variable'].isin(['neg','cam'])].value

In [9]:
wordcloud_draw(pos, "white")
wordcloud_draw(neg, "black")

NameError: name 'stopwords_set' is not defined

# topic modelling

https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

# Text preprocessing

## using spaCy

In [96]:
import spacy
!python -m spacy download es_core_news_sm
from spacy.lang.es import Spanish
from spacy.lang.es.stop_words import STOP_WORDS
nlp = spacy.load('es_core_news_sm')
parser = Spanish()

[+] Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


Tokeniza con spacy ( tambien se puede obtenet POS)

In [22]:
flatten = lambda l: [item for sublist in l for item in sublist]
doc = '. '.join(flatten(doc_all))

In [1]:
docs = nlp(doc)

NameError: name 'nlp' is not defined

In [212]:
# definir si elegir solo algunas POS


def lematiza(frase):
    '''Deveulve una lista de términos lematizados de una frase,
    además quita palabras STOP, que no sean alpha
    y las de una sola letra'''
    lemmas = []
    for token in docs:
        if token.is_punct == False and \
            token.is_stop == False and \
            token.pos_ in ['NOUN', 'PRON', 'ADJ']:
                lemmas.append(token.lemma_.lower())
    return(lemmas)

In [213]:
pos = cl_data[cl_data['variable'] == 'pos'].value.tolist()
gen = cl_data[cl_data['variable'] == 'gen'].value.tolist()
neg = cl_data[cl_data['variable'] == 'neg'].value.tolist()
cam = cl_data[cl_data['variable'] == 'cam'].value.tolist()


In [215]:
def get_lemmas(frase_lst):
    '''Devuelve una lista de listas. Realiza el
    el procesamiento de spacy y saca las lemmas'''
    lemma_list = [lematiza(frase_tag) for
                 frase_tag in [nlp(frase) for
                              frase in frase_lst]]
    return(lemma_list)

## Using wordnet de nltk

is spanish available? I think it should be possible to dl 

In [44]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/ludwig/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
get_lemma2('dogs')

'dog'

## Using udpipe

In [1]:
# This file is part of UDPipe <http://github.com/ufal/udpipe/>.
#
# Copyright 2016 Institute of Formal and Applied Linguistics, Faculty of
# Mathematics and Physics, Charles University in Prague, Czech Republic.
#
# This Source Code Form is subject to the terms of the Mozilla Public
# License, v. 2.0. If a copy of the MPL was not distributed with this
# file, You can obtain one at http://mozilla.org/MPL/2.0/.

import ufal.udpipe
# ufal.udpipe.Model etc. are SWIG-magic and cannot be detected by pylint
# pylint: disable=no-member

class Model:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()
        return output

In [6]:
pos

NameError: name 'pos' is not defined

In [2]:
model = Model('../models/spanish-gsd-ud-2.4-190531.udpipe')
sentences = model.tokenize("Hola niños como están?. Mi caso es muy complejo")

In [3]:
for s in sentences:
    model.tag(s)
    model.parse(s)

In [4]:
my_mod = model.write(sentences, 'conllu')

In [5]:
print(my_mod)

# newdoc
# newpar
# sent_id = 1
# text = Hola niños como están?.
1	Hola	hola	PART	_	_	2	amod	_	_
2	niños	niño	NOUN	_	Gender=Masc|Number=Plur	0	root	_	_
3	como	como	SCONJ	_	_	4	mark	_	_
4	están	estar	VERB	_	Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin	2	advcl	_	SpaceAfter=No
5	?	?	PUNCT	_	_	2	punct	_	SpaceAfter=No
6	.	.	PUNCT	_	_	2	punct	_	_

# sent_id = 2
# text = Mi caso es muy complejo
1	Mi	mi	DET	_	Number=Sing|Person=1|Poss=Yes|PronType=Prs	2	det	_	_
2	caso	caso	NOUN	_	Gender=Masc|Number=Sing	5	nsubj	_	_
3	es	ser	AUX	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	5	cop	_	_
4	muy	mucho	ADV	_	_	5	advmod	_	_
5	complejo	complejo	ADJ	_	Gender=Masc|Number=Sing	0	root	_	SpaceAfter=No




# topic modeling with gensim

In [80]:
from gensim import corpora

In [218]:
pos_lemmas = get_lemmas(pos)

In [219]:
dct = corpora.Dictionary(pos_lemmas)

In [220]:
from gensim.utils import simple_preprocess
# List with 2 sentences
my_docs = ["Who let the dogs out?",
           "Who? Who? Who? Who?"]

# Tokenize the docs
tokenized_list = [simple_preprocess(doc) for doc in my_docs]


In [221]:
mycorpus = [dct.doc2bow(doc, allow_update=True) for doc in pos_lemmas]

### TFIDF

In [223]:
from gensim import models
tfidf = models.TfidfModel(mycorpus, smartirs='ntc')
# com ose usa esto para el futuro??

### n-grams

In [185]:
# Build the bigram models
dataset = '. '.join(pos).split()
bigram = models.phrases.Phrases(dataset, min_count=3, threshold=10)

### topic

In [225]:
from gensim.models import LdaModel, LdaMulticore
# Step 4: Train the LDA model
lda_model = LdaMulticore(corpus=mycorpus,
                         id2word=dct,
                         random_state=100,
                         num_topics=3,
                         passes=10,
                         chunksize=1000,
                         batch=False,
                         alpha='asymmetric',
                         decay=0.5,
                         offset=64,
                         eta=None,
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         per_word_topics=True)

In [226]:
lda_model.print_topics(-1)

[(0,
  '0.026*"casar" + 0.020*"ingresar" + 0.017*"personar" + 0.017*"gente" + 0.014*"positivo" + 0.013*"enfermero" + 0.012*"profesional" + 0.012*"paciente" + 0.011*"comer" + 0.011*"hospital"'),
 (1,
  '0.018*"casar" + 0.012*"ingresar" + 0.012*"gente" + 0.010*"enfermero" + 0.009*"paciente" + 0.009*"personar" + 0.009*"hospital" + 0.008*"positivo" + 0.008*"comer" + 0.007*"coser"'),
 (2,
  '0.017*"casar" + 0.014*"ingresar" + 0.012*"hospital" + 0.012*"personar" + 0.012*"gente" + 0.010*"positivo" + 0.010*"comer" + 0.009*"profesional" + 0.009*"paciente" + 0.008*"enfermero"')]